In [1]:
import mne
from mne_connectivity import spectral_connectivity_epochs
from mne_connectivity.viz import plot_connectivity_circle
from matplotlib import pyplot as plt
import numpy as np

In [2]:
bids_dir = '/Users/hugofluhr/DATA/ds003505/'

In [3]:
from utils_hugo import read_subject_epochs

In [ ]:
epochs, y = read_subject_epochs(bids_dir=bids_dir, subject='sub-01')
metadata_list = [{'event': i} for i in y]

/Users/hugofluhr/Documents/Cours/NML/NetworkMachineLearning_2023/notebooks_hugo/utils_hugo.py:18: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  epochs = mne.read_epochs_eeglab(


Applying baseline correction (mode: mean)
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...


In [48]:
dataset = MNEDataset(epochs_list=[epochs[i] for i in range(len(epochs))],
                        metadata_list=metadata_list,
                        io_path='data/data_mne',
                        offline_transform=transforms.Compose(
                            [transforms.BandDifferentialEntropy()]),
                        online_transform=transforms.ToTensor(),
                        label_transform=transforms.Compose([
                            transforms.Select('event')
                        ]),
                        num_worker=1)

[MNE]: 100%|██████████| 588/588 [00:42<00:00, 13.70it/s]

Please wait for the writing process to complete...


In [49]:
dataset[0][1]

1

## PLI Connectivity

In [ ]:
fmin, fmax = 4., 9.
sfreq = epochs.info['sfreq']
tmin = 0.0 # exclude the baseline period

In [ ]:
con_faces = spectral_connectivity_epochs(
    epochs['Faces'], method='plv', mode='multitaper', fmin=10, fmax=50,
    faverage=True, tmin=tmin, mt_adaptive=False, n_jobs=8, verbose='CRITICAL')

In [ ]:
con_scram = spectral_connectivity_epochs(
    epochs['Scrambled'], method='plv', mode='multitaper', fmin=10, fmax=50,
    faverage=True, tmin=tmin, mt_adaptive=False, n_jobs=8, verbose='CRITICAL')

In [ ]:
A_faces = np.squeeze(con_faces.get_data('dense'))
A_faces += A_faces.T

In [ ]:
plt.imshow(A_faces, interpolation='none')
plt.colorbar()
plt.title('Faces connectivity');

In [ ]:
plt.hist(A_faces.sum(0))
plt.title('Degree distribution of faces connectivity')
plt.show()

In [ ]:
A_scram = np.squeeze(con_scram.get_data('dense'))
A_scram += A_scram.T

In [ ]:
plt.imshow(A_scram, interpolation='none')
plt.colorbar()
plt.title('Scrambled connectivity');

In [ ]:
plt.hist(A_scram.sum(0))
plt.title('Degree distribution of Scrambled connectivity')
plt.show()

In [ ]:
plt.scatter(np.triu(A_faces), np.triu(A_scram))
plt.colorbar()
plt.title('Scatter of Faces vs Scrambled for 1 subject')
plt.xlabel('Faces')
plt.ylabel('Scrambled')
plt.show()

In [ ]:
# Fancy useless plot
plot_connectivity_circle(
    con_faces.get_data(output='dense')[:, :, 0],
    epochs.ch_names)

In [ ]:
# Fancy useless plot
plot_connectivity_circle(
    con_scram.get_data(output='dense')[:, :, 0],
    epochs.ch_names)

## Enveloppe correlation connectivity

In [ ]:
from mne_connectivity import envelope_correlation

In [ ]:
env_con = envelope_correlation(epochs['Faces'])

In [ ]:
plt.matshow(env_con.get_data('dense').mean(0))
plt.colorbar()

In [ ]:
env_con_sram = envelope_correlation(epochs['Scrambled'])

In [ ]:
plt.matshow(env_con_sram.get_data('dense').mean(0))
plt.colorbar()

In [ ]:
plt.scatter(np.triu(env_con.get_data('dense').mean(0)), np.triu(env_con_sram.get_data('dense').mean(0)))
plt.colorbar()
plt.title('Scatter of Faces vs Scrambled for 1 subject')
plt.xlabel('Faces')
plt.ylabel('Scrambled')
plt.show()

In [ ]:
env_con.get_data('dense').mean(0).shape

## Compare connectivity from different epochs

In [ ]:
epochs['Faces']


In [ ]:
con_faces_1 = spectral_connectivity_epochs(
    epochs['Faces'][:10], method='plv', mode='multitaper', fmin=10, fmax=50,
    faverage=True, tmin=tmin, mt_adaptive=False, n_jobs=8, verbose='CRITICAL')

In [ ]:
con_faces_2 = spectral_connectivity_epochs(
    epochs['Faces'][-10:], method='plv', mode='multitaper', fmin=10, fmax=50,
    faverage=True, tmin=tmin, mt_adaptive=False, n_jobs=8, verbose='CRITICAL')

In [ ]:
plt.scatter(con_faces_1.get_data('dense'),con_faces_2.get_data('dense'))
plt.title('Scatter of PLV connectivity for different samples of epochs for 1 subject')
plt.show()

## Evoked response for 1 subject

In [ ]:
evoked = epochs.average(by_event_type=True)

In [ ]:
evoked[0].plot();

In [ ]:
evoked[1].plot();

In [ ]:
evoked_diff = mne.combine_evoked([evoked[1], evoked[0]], weights=[1, -1])

In [ ]:
evoked_diff.plot();